## Библиотека UDPipe
Для работы UDPipe надо поставить библиотеку  
`pip install ufal.udpipe`  
Помимо этого, нам потребуются языковые синтаксические модели, взятые отсюда 
https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3131  
Более конкретно, я буду использовать вот этот файл положив его в каталог data:  
https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/russian-syntagrus-ud-2.5-191206.udpipe?sequence=70&isAllowed=y

In [ ]:
!pip install ufal.udpipe
!pip install matplotlib
!pip install tqdm
!pip install beautifulsoup4
!pip install spacy

In [60]:
# Подключаем синтаксической анализатор и его части.
import ufal.udpipe
from ufal.udpipe import Model, Pipeline, ProcessingError

# Для отрисовки деревьев нам потребуются вот эти библиотеки.
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup as bs

from collections import defaultdict
from tqdm.auto import tqdm
import random


Загрузим уже обученную модель. 

In [61]:
model = Model.load("./data/russian-syntagrus-ud-2.5-191206.udpipe")

В состав UDPipe входят собственный токенизатор, морфология и система снятия омонимии. Создадим соответствующие объекты, а также объект, которые нам пригодятся при синтаксическом анализе.

In [62]:
# Токенизатор со снятием омонимии.
tokenizer = model.newTokenizer(model.DEFAULT)
# Объект для отображения результатов разбора.
conlluOutput = ufal.udpipe.OutputFormat.newOutputFormat("conllu")
# Объект предложения, в которое будет осуществляться разбор.
sentence = ufal.udpipe.Sentence()
# Объект для ошибок, возникающих в ходе разбора.
error = ufal.udpipe.ProcessingError()

Теперь нам необходимо передать токенизатору текст, который мы собираемся разбирать, при помощи функции `setText`. 

In [63]:
tokenizer.setText("Мама мыла раму. При этом Рама краснел со сраму. ");

Выбирать очередное предложение можно при помощи функции `nextSentence`. Эта функция возвращает `True`, если предложение извлеклось, и `False`, если предложений в тексте больше нет.

Далее предложение передается на морфологическую разметку со снятием омонимии (тэггинг) и синтаксический анализ (парсинг).

Результаты разбора можно отображать в разных форматах, но мы будем использовать формат conllu.

In [64]:
# Токенизация.
tokenizer.nextSentence(sentence, error)
# Морфологичекий анализ со снятием омонимии.
model.tag(sentence, model.DEFAULT)
# Синтаксический анализ.
model.parse(sentence, model.DEFAULT)
# Отображение результатов.
print(conlluOutput.writeSentence(sentence))

# newdoc
# newpar
# sent_id = 1
# text = Мама мыла раму.
1	Мама	мама	NOUN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	2	nsubj	_	_
2	мыла	мыть	VERB	_	Aspect=Imp|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
3	раму	рама	NOUN	_	Animacy=Anim|Case=Acc|Gender=Fem|Number=Sing	2	obj	_	SpaceAfter=No
4	.	.	PUNCT	_	_	2	punct	_	_




Выберем следующее предложение и проведем его анализ.

In [65]:
tokenizer.nextSentence(sentence, error)

model.tag(sentence, model.DEFAULT)
model.parse(sentence, model.DEFAULT)

print(conlluOutput.writeSentence(sentence))

# sent_id = 2
# text = При этом Рама краснел со сраму.
1	При	при	ADP	_	_	2	case	_	_
2	этом	это	PRON	_	Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing	4	obl	_	_
3	Рама	рама	PROPN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	4	nsubj	_	_
4	краснел	краснеть	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
5	со	с	ADP	_	_	6	case	_	_
6	сраму	срам	NOUN	_	Animacy=Inan|Case=Par|Gender=Masc|Number=Sing	4	obl	_	SpaceAfter=No
7	.	.	PUNCT	_	_	4	punct	_	_




Объект `Sentence` содержит в себе свойство `words`, являющееся списком слов. Каждое слово включает в себя, среди прочего, следующие свойства:
- lemma - начальная форма слова,
- form - токен, 
- head - номер родительского элемента в списке слов,
- deprel - название отношения,
- upostag - часть речи,
- feats - грамматические параметры,
- misc - дополнительные параметры.

In [66]:
for i, word in enumerate(sentence.words):
    print(i, "->", word.head, word.lemma, "[", word.form, "]", word.upostag, word.feats, word.deprel, word.misc)

0 -> -1 <root> [ <root> ] <root> <root>  
1 -> 2 при [ При ] ADP  case 
2 -> 4 это [ этом ] PRON Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing obl 
3 -> 4 рама [ Рама ] PROPN Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing nsubj 
4 -> 0 краснеть [ краснел ] VERB Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act root 
5 -> 6 с [ со ] ADP  case 
6 -> 4 срам [ сраму ] NOUN Animacy=Inan|Case=Par|Gender=Masc|Number=Sing obl SpaceAfter=No
7 -> 4 . [ . ] PUNCT  punct 


Напишем функцию, которая будет выводить предложение в формате дерева. Все потомки следуют за родителем, перед потомком относительно родителя дается отступ.

In [67]:
def print_sent(sentence, space="", head=0):
    for i, word in enumerate(sentence.words):
        if word.head == head:
            print(space, i, word.form)
            print_sent(sentence, space+"    ", i)

print_sent(sentence)

 4 краснел
     2 этом
         1 При
     3 Рама
     6 сраму
         5 со
     7 .


Давайте возьмем новость и выделим из нее все сочетания "существительное + существительное в родительном падеже". Такие сочетания интересны тем, что могут претендовать на роль терминов.

In [68]:
# Загружаем новость и получаем её текст.
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')
souped = bs(page.text)
text = '\n'.join([p.get_text() for p in souped.find_all("div", attrs={'class': 'topic-body'})[0]("p")])
text

'Анатолий Жданов / «Коммерсантъ»\nУченые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.\nОпыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.\nПо словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.\nРанее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и поэтому яблоко желательно съедать вместе с огр

In [69]:
def find_child_Ng(sentence, parent):
    """ Функция поиска всех потомков вершины с номером parent, являющихся
    существительными в родительном падеже.
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.head == parent and word.upostag == "NOUN" and "Case=Gen" in word.feats:
            children.append(i)
    return children

def find_NNg(sentence, combinations):
    """ Функция поиска сочетаний "существительное + существительное в родительном падеже".
    """
    for i, word in enumerate(sentence.words):
        if word.upostag == "NOUN":
            children = find_child_Ng(sentence, i)
            if children != []:
                #print(word.lemma, [sentence.words[k].form for k in children])
                for child in children:
                    combinations[word.lemma][sentence.words[child].form] += 1


Проанализируем все предложения текста и выделим сочетания.

In [70]:
from pprint import pprint
tokenizer.setText(text) # Установить текст.

combinations = defaultdict(lambda:defaultdict(int))

while tokenizer.nextSentence(sentence, error): # Полчаем следующее предложение.
    model.tag(sentence, model.DEFAULT) # Токенизация.
    model.parse(sentence, model.DEFAULT) # Синтаксический анализ.
    find_NNg(sentence, combinations) # Поиск сочетаний.
    
pprint(combinations)

defaultdict(<function <lambda> at 0x1247dfec0>,
            {'большинство': defaultdict(<class 'int'>, {'людей': 1}),
             'клетка': defaultdict(<class 'int'>, {'мозга': 1}),
             'концентрация': defaultdict(<class 'int'>, {'фитонутриентов': 1}),
             'мозг': defaultdict(<class 'int'>, {'мышей': 1}),
             'образование': defaultdict(<class 'int'>, {'нейронов': 1}),
             'процент': defaultdict(<class 'int'>, {'веществ': 1}),
             'работа': defaultdict(<class 'int'>, {'органов': 1, 'мозга': 1}),
             'результат': defaultdict(<class 'int'>, {'исследования': 1}),
             'сердцевина': defaultdict(<class 'int'>, {'фрукта': 1}),
             'слово': defaultdict(<class 'int'>, {'ученых': 1}),
             'употребление': defaultdict(<class 'int'>, {'яблок': 1}),
             'ученый': defaultdict(<class 'int'>, {'университета': 1}),
             'центр': defaultdict(<class 'int'>, {'заболеваний': 1}),
             'эффект': defaultd

Пусть к нам в руки попал фрагмент новостей с Ленты.ру за 2018 год. Посмотрим какие сочетания можно извлечь из него.

In [74]:
with open("./data/lenta2018.txt", encoding="utf-8") as news_file: # Файл с новостями.
    text_news = [n.split("-----\n")[1] for n in news_file.read().split("=====\n")[1:]]
    

In [75]:
# Проверим, что новости зугрузились.
text_news[0]

'Пользователи Reddit выразили симпатию российским гопникам. Посетители ресурса оценили фотографию с молодыми людьми, присевшими, как львы-стражи, на корточки на опорные столбы ворот. Юзеры вывели снимок в топ ресурса.Разместивший фото пользователь с ником GAZAYOUTH93X назвал композицию «Дмитрий и Анатолий: привратники и стражи Славатара».«Молитесь, чтобы вам удалось пройти через адидассассинов!» — шутит joekillsshit, намекая на сюжет игры Assassin’s Creed.Пользователи иронизируют, что рады, наконец, увидеть, как выглядят «древние славяне» из города Славатар.Материалы по теме00:05 — 21 января 2017«Пацаны вообще ребята»Как русские гопники покорили интернет и научили Запад сидеть на кортахОднажды повстречавшиеся с гопниками из России комментаторы отмечают их крутой нрав и просят посоветовать, как защититься от них. «Каждый раз, когда я пытался пройти мимо них, я в считанные секунды становился недееспособным, потому что они пинали меня до смерти, крича "CYKA *** !" снова и снова», — пишет 

In [76]:
text_news = text_news

In [77]:
combinations = defaultdict(lambda:defaultdict(int))

for text in tqdm(text_news):
    tokenizer.setText(text) # Установить текст.
    while tokenizer.nextSentence(sentence, error): # Полчаем следующее предложение.
        model.tag(sentence, model.DEFAULT) # Токенизация.
        model.parse(sentence, model.DEFAULT) # Синтаксический анализ.
        find_NNg(sentence, combinations) # Поиск сочетаний.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1707/1707 [04:16<00:00,  6.66it/s]


In [78]:
for parent, childs in combinations.items():
    for child, freq in childs.items():
        if freq > 30:
            print(parent, child, freq)

сентябрь года 39
глава государства 56
тысяча долларов 39
тысяча рублей 54
тысяча человек 33
статья УК 51
лишение свободы 31
декабрь года 86
октябрь года 37
член экипажа 52
миллион долларов 64
миллион рублей 59
нарушение правил 65
миллиард долларов 42
рубль миллиона 37
ноябрь года 47
деятельность террористов 32
примечание «Ленты.ру» 47
решение суда 37
январь года 57
министр обороны 31
конец января 42
конец года 58
начало года 32
февраль года 41
пассажир членов 35
доллар миллиарда 48
церемония открытия 35


При анализе текстов существует задача оценки тональности текста. Тональность текста определяет какого роде слова содержатся в тексте - негативные или позитивные. Например, если у нас имеется отзыв на какой-то товар, услугу или организацию, мы можем определить, положительный это отзыв или отрицательный. Для оценки тональности отзыва используют, среди прочего, словари оценочной лексики.

Один отзыв может содержать как критику, так и похвалу, в зависимости от того, что в нем рассматривается (Прекрасное обслуживание, но пироженые были несвежими). Код ниже использует словарь оценочной лексики для того, чтобы понять, какие глаголы сочетаются чаще с отрицательно окрашенными словами, а какие - с положительными.

In [84]:
# Загружаем словарь оценочной лексики.
with open("./data/rusentilex_2017.txt", encoding="utf-8") as senti_file: 
    for i in range(18): # Пропускаем заголовок файла.
        _ = senti_file.readline()
        
    senti_words = {line.split(", ")[0]: line.split(", ")[3] for line in senti_file.readlines()}

In [85]:
# Посмотрим на первые 10 слов словаря.
list(senti_words.items())[:10]

[('аборт', 'negative'),
 ('абортивный', 'negative'),
 ('абракадабра', 'negative'),
 ('абсурд', 'negative'),
 ('абсурдность', 'negative'),
 ('абсурдный', 'negative'),
 ('авантюра', 'negative'),
 ('авантюризм', 'negative'),
 ('авантюрист', 'negative'),
 ('авантюристический', 'negative')]

In [86]:
def find_childs(sentence, parent):
    """ Функция для выделения потомков вершины с номером parent из предложения sentence в формате CONLLU.
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.head == parent:
            children.append(i)
    return children

def find_sentiments(sentence, senti_words, sentiments):
    """ Ищет слова оценочной лексики senti_words в предложении sentence.
        Найденные пары возвращает в список sentiments.
    """
    for i, word in enumerate(sentence.words):
        children = find_childs(sentence, i)
        for child in children:
            if sentence.words[child].lemma in senti_words.keys():
                if word.lemma not in sentiments.keys():
                    sentiments[word.lemma] = defaultdict(int)
                sentiments[word.lemma][senti_words[sentence.words[child].lemma]] += 1


In [ ]:
sentiments = {}

for sent_text in tqdm(text_news):
    tokenizer.setText(sent_text)
    while tokenizer.nextSentence(sentence, error):
        model.tag(sentence, model.DEFAULT)
        model.parse(sentence, model.DEFAULT)
        find_sentiments(sentence, senti_words, sentiments)


In [83]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>50 and senti!='<root>'], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>50 and senti!='<root>']

([('получить',
   defaultdict(int,
               {'negative': 56,
                'positive': 33,
                'neutral': 3,
                'positive/negative': 3})),
  ('стать',
   defaultdict(int,
               {'negative': 79,
                'positive': 59,
                'neutral': 32,
                'positive/negative': 6})),
  ('признать',
   defaultdict(int,
               {'negative': 66,
                'positive/negative': 2,
                'positive': 10,
                'neutral': 3})),
  ('мочь',
   defaultdict(int,
               {'negative': 76,
                'positive': 51,
                'positive/negative': 7,
                'neutral': 16})),
  ('дело', defaultdict(int, {'negative': 144, 'neutral': 2, 'positive': 3})),
  ('произойти',
   defaultdict(int,
               {'negative': 82,
                'neutral': 5,
                'positive/negative': 3,
                'positive': 4})),
  ('группировка',
   defaultdict(int, {'negative': 54, 'neutral': 4

Посмотрим какие слова чаще встречаются с о словами негатитвной окраски, а какие - с позитивными.

## Синтаксический анализатор SpaCy

Ещё одним синтаксическим анализатором является Spacy. Здесь мы работаем с версией 3, которая вышла в марте 2021 года. Библиотеки можно поставить при помощи pip, а **языковые модели** для нее можно взять [здесь](https://github.com/buriy/spacy-ru) (хотя проще скачать модель в wheel-формате и тоже поставить при помощи pip).

In [93]:
# Импортируемм нужные библиотеки.
import spacy
from spacy import displacy

In [ ]:
!python -m spacy download ru_core_news_sm

In [95]:
# Загружаем языковую модель.
nlp = spacy.load("ru_core_news_sm")

Дерево в Spacy представляется именно в виде дерева зависимостей: как совокупность вершин и связей между ними. Каждая вершина соответствует слову из предложения, у слова есть лемма (поле `lemma_`) и токен (поле `text`).

In [96]:
# Передаем текст новости в Spacy для разбора, получаем объект документа с результатами разбора.
doc = nlp(text_news[0])
# Перебираем предложения в тексте. 
# Но так как мне нужны первые два предложения, я превращаю итерируемый объект sents в список.
for s in list(doc.sents)[:2]:
    print(list([f'"{t.lemma_}" - "{t.text}"' for t in s]))

['"пользователь" - "Пользователи"', '"reddit" - "Reddit"', '"выразить" - "выразили"', '"симпатия" - "симпатию"', '"российский" - "российским"', '"гопник" - "гопникам"', '"." - "."']
['"посетитель" - "Посетители"', '"ресурс" - "ресурса"', '"оценить" - "оценили"', '"фотография" - "фотографию"', '"с" - "с"', '"молодой" - "молодыми"', '"человек" - "людьми"', '"," - ","', '"присесть" - "присевшими"', '"," - ","', '"как" - "как"', '"лев" - "львы"', '"-" - "-"', '"страж" - "стражи"', '"," - ","', '"на" - "на"', '"корточки" - "корточки"', '"на" - "на"', '"опорный" - "опорные"', '"столб" - "столбы"', '"ворот" - "ворот"', '"." - "."']


Вместе со Spacy ставится библиотека dasplaycy, которая умеет отображать текст в виде деревьев зависимости. Посмотрим на первые два предложения.

In [97]:
for s in list(doc.sents)[:2]:
    displacy.render(s, style="dep", minify=True, jupyter=True, options={"distance":90})

Для предложения определено свойство `root`, которое является корневой вершиной предложения.  
Помимо леммы и токена, каждая вершина хранит список потомков `children`, список родителей `ancestors`, вид зависимости (роль слова при родителе) `dep_`, часть речи `tag_`.

In [98]:
sent0 = list(doc.sents)[0]
childs0 = list(sent0.root.children)
print(sent0.root, childs0)
print(childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0])

выразили [Пользователи, симпатию, гопникам, .]
пользователь Пользователи nsubj NOUN -> выразили


Решим ту же задачу с поиском эмоционально окрашенных слов в новостях.

In [99]:
# Поиск эмоионально окрашенных потомков для SpaCy.
def find_sentiments_spacy(node, senti_words, sentiments):
    for child in node.children:
        if child.lemma_ in senti_words.keys():
            if node.lemma_ not in sentiments.keys():
                sentiments[node.lemma_] = defaultdict(int)
            sentiments[node.lemma_][senti_words[child.lemma_]] += 1


In [100]:
# Перебираем все прдложения первой новости и ищем эмоционально окрашенные слова.
sentiments = {}
for sent in doc.sents:
    find_sentiments_spacy(sent.root, senti_words, sentiments)
sentiments

{'выразить': defaultdict(int, {'positive': 1, 'negative': 1}),
 '«молитесь': defaultdict(int, {'positive': 1}),
 'покорить': defaultdict(int, {'negative': 1}),
 'становиться': defaultdict(int, {'negative': 1}),
 'продемонстрировать': defaultdict(int, {'positive': 1, 'negative': 1}),
 'обрести': defaultdict(int, {'positive': 1})}

In [ ]:
# Найдем все эмоционально окрашенные фразы во всех новостях.
sentiments = {}
for sent_text in tqdm(text_news):
    doc = nlp(sent_text)
    for sent in doc.sents:
        find_sentiments_spacy(sent.root, senti_words, sentiments)


 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                | 1567/1707 [02:53<00:13, 10.09it/s]

Как вы можете видеть, SpaCy работает быстрее (так как я использовал модель с повышеной скоростью, а не с повышенной точностью), но результаты работы несколько отличаются. Это означает, что SpaCy возвращает деревья другой структуры.

In [40]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>30], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>30]

([('признать',
   defaultdict(int, {'negative': 48, 'positive': 8, 'neutral': 2})),
  ('стать',
   defaultdict(int,
               {'negative': 40,
                'positive': 35,
                'neutral': 17,
                'positive/negative': 1})),
  ('мочь',
   defaultdict(int,
               {'negative': 36,
                'positive': 19,
                'neutral': 11,
                'positive/negative': 3,
                'пострадать': 1})),
  ('произойти',
   defaultdict(int,
               {'negative': 58,
                'positive/negative': 2,
                'neutral': 5,
                'positive': 1}))],
 '\n----\n',
 [('принять', defaultdict(int, {'positive': 38, 'neutral': 4, 'negative': 5})),
  ('обратить',
   defaultdict(int, {'positive': 63, 'neutral': 1, 'negative': 2})),
  ('стать',
   defaultdict(int,
               {'negative': 40,
                'positive': 35,
                'neutral': 17,
                'positive/negative': 1}))])